# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/keertanachandar/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/keertanachandar/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Use-Case Data!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/keertanachandar/Public/Github/code/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/keertanachandar/Public/Github/code/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/keertanachandar/Public/Github/code/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 64, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node 'e1d00d'. Skipping!
Property 'summary' already exists in node '0a6301'. Skipping!
Property 'summary' already exists in node 'eb90b0'. Skipping!
Property 'summary' already exists in node '3c80db'. Skipping!
Property 'summary' already exists in node '4f5a26'. Skipping!
Property 'summary' already exists in node 'ac6407'. Skipping!
Property 'summary' already exists in node '21f51f'. Skipping!
Property 'summary' already exists in node '0b0588'. Skipping!
Property 'summary' already exists in node 'b57508'. Skipping!
Property 'summary' already exists in node '3cc62a'. Skipping!
Property 'summary' already exists in node '3d03a3'. Skipping!
Property 'summary' already exists in node '3fe9c6'. Skipping!
Property 'summary' already exists in node '9d480b'. Skipping!
Property 'summary' already exists in node '1b582e'. Skipping!
Property 'summary' already exists in node '6c3e7d'. Skipping!
Property 'summary' already exists in node '659be2'. Skipping!
Property

Applying CustomNodeFilter:   0%|          | 0/8 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/48 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '3d03a3'. Skipping!
Property 'summary_embedding' already exists in node '21f51f'. Skipping!
Property 'summary_embedding' already exists in node 'b57508'. Skipping!
Property 'summary_embedding' already exists in node 'ac6407'. Skipping!
Property 'summary_embedding' already exists in node '3c80db'. Skipping!
Property 'summary_embedding' already exists in node 'e1d00d'. Skipping!
Property 'summary_embedding' already exists in node '3cc62a'. Skipping!
Property 'summary_embedding' already exists in node '4f5a26'. Skipping!
Property 'summary_embedding' already exists in node '0b0588'. Skipping!
Property 'summary_embedding' already exists in node 'eb90b0'. Skipping!
Property 'summary_embedding' already exists in node '0a6301'. Skipping!
Property 'summary_embedding' already exists in node '3fe9c6'. Skipping!
Property 'summary_embedding' already exists in node '9d480b'. Skipping!
Property 'summary_embedding' already exists in node 'ea51ed'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 86, relationships: 711)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 86, relationships: 711)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,O*NET what is it,[Introduction ChatGPT launched in November 202...,The provided context does not include informat...,single_hop_specifc_query_synthesizer
1,What US?,[Table 1: ChatGPT daily message counts (millio...,The context discusses the US in relation to Ch...,single_hop_specifc_query_synthesizer
2,Could you explain the significance of SOC2 cod...,[Variation by Occupation Figure 23 presents va...,The provided context does not include specific...,single_hop_specifc_query_synthesizer
3,How does the use of Writing in ChatGPT relate ...,[Conclusion This paper studies the rapid growt...,The context indicates that the study on ChatGP...,single_hop_specifc_query_synthesizer
4,How do the variations in ChatGPT usage by occu...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,The first segment highlights how ChatGPT usage...,multi_hop_abstract_query_synthesizer
5,so like how does the variation in work activit...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,The context shows that users in different occu...,multi_hop_abstract_query_synthesizer
6,Based on the data indicating that nearly half ...,[<1-hop>\n\nConclusion This paper studies the ...,The data showing that nearly half of all messa...,multi_hop_abstract_query_synthesizer
7,"How does the growth of Claude, as discussed in...",[<1-hop>\n\nTable 1: ChatGPT daily message cou...,The context indicates that ChatGPT experienced...,multi_hop_specific_query_synthesizer
8,Based on the data from OpenAI regarding ChatGP...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,"The rapid adoption of ChatGPT, with 18 billion...",multi_hop_specific_query_synthesizer
9,How does the usage of ChatGPT in the US relate...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,"In the US, ChatGPT usage has shown significant...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node 'e1d5ba'. Skipping!
Property 'summary' already exists in node '16c8e8'. Skipping!
Property 'summary' already exists in node 'dff4f8'. Skipping!
Property 'summary' already exists in node '8b7534'. Skipping!
Property 'summary' already exists in node 'd56df7'. Skipping!
Property 'summary' already exists in node 'e382cd'. Skipping!
Property 'summary' already exists in node 'fde169'. Skipping!
Property 'summary' already exists in node '250f55'. Skipping!
Property 'summary' already exists in node '540856'. Skipping!
Property 'summary' already exists in node 'c5aad1'. Skipping!
Property 'summary' already exists in node '486f0c'. Skipping!
Property 'summary' already exists in node 'aa8cce'. Skipping!
Property 'summary' already exists in node 'a73c02'. Skipping!
Property 'summary' already exists in node 'fdf00f'. Skipping!
Property 'summary' already exists in node '722d30'. Skipping!
Property 'summary' already exists in node '854869'. Skipping!
Property

...

Property 'summary_embedding' already exists in node 'fdf00f'. Skipping!
Property 'summary_embedding' already exists in node '540856'. Skipping!
Property 'summary_embedding' already exists in node '250f55'. Skipping!
Property 'summary_embedding' already exists in node '16c8e8'. Skipping!
Property 'summary_embedding' already exists in node 'dff4f8'. Skipping!
Property 'summary_embedding' already exists in node 'fde169'. Skipping!
Property 'summary_embedding' already exists in node 'e1d5ba'. Skipping!
Property 'summary_embedding' already exists in node 'c5aad1'. Skipping!
Property 'summary_embedding' already exists in node '722d30'. Skipping!
Property 'summary_embedding' already exists in node 'e382cd'. Skipping!
Property 'summary_embedding' already exists in node '8b7534'. Skipping!
Property 'summary_embedding' already exists in node 'd56df7'. Skipping!
Property 'summary_embedding' already exists in node 'aa8cce'. Skipping!
Property 'summary_embedding' already exists in node 'a73c02'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What insights does Wiggers (2025) provide rega...,[Introduction ChatGPT launched in November 202...,"Wiggers (2025) reports that by July 2025, 18 b...",single_hop_specifc_query_synthesizer
1,What is the significance of June 2024?,[Total daily counts are exact measurements of ...,The context reports on message volume data end...,single_hop_specifc_query_synthesizer
2,What is the role of Engineering and science in...,[Variation by Occupation Figure 23 presents va...,The context indicates that users in engineerin...,single_hop_specifc_query_synthesizer
3,How does expressing relate to ChatGPT usage in...,[Conclusion This paper studies the rapid growt...,The provided context does not include specific...,single_hop_specifc_query_synthesizer
4,Considering the rapid launch and widespread ad...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"Since the launch of ChatGPT in November 2022, ...",multi_hop_abstract_query_synthesizer
5,How do the societal effects of AI relate to us...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The rapid growth and widespread adoption of Ch...,multi_hop_abstract_query_synthesizer
6,"H0w does AI, like ChatGPT, impact economic gro...",[<1-hop>\n\nIntroduction ChatGPT launched in N...,The context explains that the rapid growth and...,multi_hop_abstract_query_synthesizer
7,How do user demographics and usage patterns re...,[<1-hop>\n\nConclusion This paper studies the ...,"The data shows that ChatGPT's rapid growth, wi...",multi_hop_abstract_query_synthesizer
8,Considering the rapid growth of ChatGPT's usag...,[<1-hop>\n\nTotal daily counts are exact measu...,The second-hop context highlights that since C...,multi_hop_specific_query_synthesizer
9,How does OpenAI's development and widespread a...,[<1-hop>\n\nTotal daily counts are exact measu...,"The context highlights that ChatGPT, developed...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [16]:
from langsmith import Client

client = Client()

dataset_name = "Use Case Synthetic Data - AIE8"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [17]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [18]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [20]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [21]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG"
)

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [23]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [25]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [26]:
rag_chain.invoke({"question" : "What are people doing with AI these days?"})

'Based on the provided context, people are using AI, particularly generative AI like ChatGPT, in a variety of ways both at work and outside of work. AI is used to perform workplace tasks by either augmenting or automating human labor. It helps with producing writing, software code, spreadsheets, and other digital products, distinguishing it from traditional web search engines. Users interact with AI with intents that include asking for information or advice, doing tasks (producing output), and expressing themselves. Generative AI is flexible and used across many different purposes, including professional work, self-expression, and creative uses such as games and role play. Additionally, AI serves as co-workers producing outputs or as co-pilots improving human problem-solving productivity.\n\nIn summary, people nowadays are using AI to:\n\n- Augment or automate workplace tasks  \n- Produce written content, code, and digital products  \n- Seek information and advice  \n- Engage in self-e

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [27]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [28]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this response dope, lit, cool, or is it just a generic response?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dopeness_evaluator`:

## LangSmith Evaluation

In [29]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'memorable-building-53' at:
https://smith.langchain.com/o/705da240-cb86-4803-963a-cc24089b4e94/datasets/0415c741-517a-4245-b240-bc248b070eb5/compare?selectedSessions=c4d82b35-f8f5-47eb-a6cd-7435dceae1da




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How does Collis relate to the practical guidan...,I don't know.,None,"The context mentions Collis twice, emphasizing...",0,0,0,0.721190,1c43f9b8-73e5-42e9-8509-075c6b59f1c7,234e4963-26b8-4290-b431-04ea44b0311c
1,how does brynjolfsson relate to chatgpt's grow...,Brynjolfsson is mentioned in the context of es...,None,The context mentions Collis and Brynjolfsson (...,1,1,0,1.460609,8c679bb5-ce3d-4a2a-9389-c23d5959d1d4,ea4fc0bd-91ed-4007-a7b7-972347401897
2,How does OpenAI's development and widespread a...,OpenAI's development and widespread adoption o...,None,"The context highlights that ChatGPT, developed...",1,1,0,4.579385,34d59642-7f1c-46b3-8b59-d63fac0178ef,dbe41cde-d7d3-499e-866b-c8c2364a5ada
3,Considering the rapid growth of ChatGPT's usag...,"Based on the provided context, OpenAI’s develo...",None,The second-hop context highlights that since C...,1,1,0,6.294479,5c65d21c-26a6-4be2-a3f9-f2fcbbe1b585,29e58047-d9bd-45e7-8fd5-48bf0d87ff47
4,How do user demographics and usage patterns re...,"Based on the provided context, ChatGPT experie...",None,"The data shows that ChatGPT's rapid growth, wi...",0,0,0,3.399520,cd77ff84-9bb5-4ed3-8fec-91e4758e4fbd,6fcb0d57-6168-497e-8678-b4a45a356d96
5,"H0w does AI, like ChatGPT, impact economic gro...","Based on the provided context, AI like ChatGPT...",None,The context explains that the rapid growth and...,1,0,0,3.648591,ba5218de-f5af-4f87-9044-72c08567d54a,436c964c-1937-4af2-a7a9-da1bc882a9b9
6,How do the societal effects of AI relate to us...,"The societal effects of AI, as reflected in th...",None,The rapid growth and widespread adoption of Ch...,1,1,0,3.874941,39a160bd-0fcf-4186-97d1-5a9abcc055a2,84e3ce15-6abe-4923-9d10-4d07f92e5db6
7,Considering the rapid launch and widespread ad...,"Based on the provided context, the impact of A...",None,"Since the launch of ChatGPT in November 2022, ...",1,1,0,5.768067,0d58a781-a44b-4ebc-949a-a9c0eb033ed7,a2871785-f042-415d-89f9-37da01ced50e
8,How does expressing relate to ChatGPT usage in...,"Based on the provided context, very few work-r...",None,The provided context does not include specific...,0,0,0,1.384976,4ddf63d8-0b58-46b9-a7a4-679a8ca0f497,759adf34-3e1f-43e9-9272-aeccddb4a35d
9,What is the role of Engineering and science in...,I don't know.,None,The context indicates that users in engineerin...,0,0,0,0.572657,66c38e65-cb9f-46f0-8b6f-e8508444746d,f2799a3d-200d-4b12-8391-797a0029ca75


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [30]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [31]:
rag_documents = docs

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [33]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [34]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [35]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [36]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [37]:
dopeness_rag_chain.invoke({"question" : "How are people using AI to make money?"})

"Alright, let's crank this up to eleven: People are not just letting AI do their grunt work—they're turning ChatGPT into a *digital sidekick* that fuels their hustle. According to the slick insights from Collis and Brynjolfsson (2025), folks leverage ChatGPT as an advisor and research assistant, supercharging their decision-making mojo in knowledge-heavy jobs. This isn't about AI botching tasks for you; it’s about AI cranking up your brainpower, helping you make smarter, sharper calls that boost productivity and cash flow.\n\nSo, the magic sauce? Using AI to *augment* human intelligence—providing crisp decision support—makes the grind smoother and $$ greener. It’s about stacking AI on top of your own skills, not replacing you, and that combo rakes in that sweet surplus, estimated at a staggering $97 billion a year just in the US. That's people tapping AI to turn knowledge into income like absolute bosses. Boom."

Finally, we can evaluate the new chain on the same test set!

In [38]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'extraneous-rest-15' at:
https://smith.langchain.com/o/705da240-cb86-4803-963a-cc24089b4e94/datasets/0415c741-517a-4245-b240-bc248b070eb5/compare?selectedSessions=9a4ed3fb-bab6-4f16-ba25-402917a54d86




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How does Collis relate to the practical guidan...,"Yo, the docs don’t drop a single beat about Co...",None,"The context mentions Collis twice, emphasizing...",0,0,1,1.425442,1c43f9b8-73e5-42e9-8509-075c6b59f1c7,70b69028-ad12-4d35-8fd5-dd8821521f8e
1,how does brynjolfsson relate to chatgpt's grow...,"Yo, here’s the deal—based on the slick context...",None,The context mentions Collis and Brynjolfsson (...,0,0,1,3.669853,8c679bb5-ce3d-4a2a-9389-c23d5959d1d4,f3dd81fd-065e-42b2-8319-b55d26f3f1df
2,How does OpenAI's development and widespread a...,"Alright, here’s the lowdown dripping in pure d...",None,"The context highlights that ChatGPT, developed...",1,1,1,4.545461,34d59642-7f1c-46b3-8b59-d63fac0178ef,64efad6a-4ad6-45a8-a342-f94ad45723aa
3,Considering the rapid growth of ChatGPT's usag...,"Yo, buckle up—because OpenAI’s LLM wizardry wi...",None,The second-hop context highlights that since C...,1,1,1,7.064147,5c65d21c-26a6-4be2-a3f9-f2fcbbe1b585,d2326e51-70fd-45ab-90af-4d2c37f34287
4,How do user demographics and usage patterns re...,"Oh, buckle up because this is a turbocharged A...",None,"The data shows that ChatGPT's rapid growth, wi...",1,1,1,9.836313,cd77ff84-9bb5-4ed3-8fec-91e4758e4fbd,dfc94936-0c7d-4283-a32c-20855c6bba46
5,"H0w does AI, like ChatGPT, impact economic gro...","Alright, let’s crank this up to eleven and dec...",None,The context explains that the rapid growth and...,1,1,1,5.364975,ba5218de-f5af-4f87-9044-72c08567d54a,541835d8-ecd1-4876-b18b-5d83b76a70cd
6,How do the societal effects of AI relate to us...,"Alright, let’s unpack this AI saga with some s...",None,The rapid growth and widespread adoption of Ch...,1,1,1,5.148599,39a160bd-0fcf-4186-97d1-5a9abcc055a2,5a343618-fb60-480b-af3f-a85de2383ee8
7,Considering the rapid launch and widespread ad...,"Alright, strap in for the dopest AI scoop on h...",None,"Since the launch of ChatGPT in November 2022, ...",1,1,1,8.413634,0d58a781-a44b-4ebc-949a-a9c0eb033ed7,280c54ec-cd60-419b-ae45-d3128ebf65a1
8,How does expressing relate to ChatGPT usage in...,"Alright, here’s the ultra-dopeness on **Expres...",None,The provided context does not include specific...,0,0,1,3.686550,4ddf63d8-0b58-46b9-a7a4-679a8ca0f497,ea3694e1-e947-4f81-99b3-c6d595fddb79
9,What is the role of Engineering and science in...,"Alright, here’s the skinny on Engineering and ...",None,The context indicates that users in engineerin...,1,0,1,4.489990,66c38e65-cb9f-46f0-8b6f-e8508444746d,0419e36e-9766-486f-85c6-8d97739ef4f7


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.